# PROJETO 1 - ANALISE DE ALGORITMO

In [1]:
using Plots
using Statistics
using Random

## VECTOR GENERATOR - SIZE(N & Q)

In [2]:
# Function to generate a vector of random values
function generate_vectors(size::Int, upper_limit::Int)
    return rand(1:upper_limit, size)
end

generate_vectors (generic function with 1 method)

In [3]:
# Function to generate a random key
function generate_keys(size::Int, upper_limit::Int)
    return rand(1:upper_limit, size)
end

generate_keys (generic function with 1 method)

## SEARCH FUNTIONS

In [4]:
#Dummy function to measure time and execute search function
function dummy_search(search_function::Function, vector, keys)
    initial_time = time_ns()
    for key in keys
        search_function(vector, keys)
    end
    final_time = time_ns()
    return (final_time - initial_time) / 1e9
end

dummy_search (generic function with 1 method)

In [5]:
# Simple linear search function
function simple_search(vector, key)
    for i in 1:length(vector)
        if vector[i] == key
            return i  # Returns the index where the key was found
        end
    end
    return -1  # Returns -1 if the key is not found
end

simple_search (generic function with 1 method)

In [6]:
# Optimized linear search function
function optimized_search(sorted_vector, key)
    for i in 1:length(sorted_vector)
        if key == sorted_vector[i]
            return i  # Returns the index where the key was found
        elseif key < sorted_vector[i]
            return -1  # Returns -1 if the key is not found
        end
    end
    return -1  # Returns -1 if the key is not found
end

optimized_search (generic function with 1 method)

In [7]:
# Binary search function
function binary_search(sorted_vector, key)
    low = 1
    high = length(sorted_vector)

    while low <= high
        mid = (low + high) ÷ 2
        if sorted_vector[mid] == key
            return mid  # Returns the index where the key was found
        elseif sorted_vector[mid] < key
            low = mid + 1
        else
            high = mid - 1
        end
    end
    return -1  # Returns -1 if the key is not found
end

binary_search (generic function with 1 method)

## MAIN - BENCHMARK AND PLOTS

In [8]:
# VARS
upper_limit = 1000 # Upper limit for generating random numbers
n_values = [10^4, 10^5, 10^6, 10^7]  # Vectors Sizes
q_values = [10^2, 10^3, 10^4, 10^5]  # Amount of Keys

# Initialize the list of vectors and keys
vector_list = [generate_vectors(n, upper_limit) for n in n_values]
key_list = [generate_keys(q, upper_limit) for q in q_values]

# Initialize the list of timers
time_simple_search = Matrix{Float64}(undef, length(n_values), length(q_values))
time_optimized_search = Matrix{Float64}(undef, length(n_values), length(q_values))
time_binary_search = Matrix{Float64}(undef, length(n_values), length(q_values))
time_to_sort = Vector{Float64}(undef, length(n_values))

4-element Vector{Float64}:
 1.5611e-319
 7.74682412640213e-304
 5.43230922614e-312
 1.09231905977e-311

In [9]:
# Simple linear search - Benchmarking
for i in 1:length(n_values)  # For each vector size
    for j in 1:length(q_values)  # For each key numbers
        time_simple_search[i, j] = dummy_search(simple_search, vector_list[i], key_list[j])
    end
end

In [10]:
# Sorting the vectors - Benchmarking
for i in 1:length(n_values)   # For each vector size
    initial_time = time_ns()
    vector_list[i] = sort(vector_list[i])  # Sort the vectors
    final_time = time_ns()
    time_to_sort[i] = (final_time - initial_time) / 1e9
end

In [11]:
# Optimized Linear search - Benchmarking
for i in 1:length(n_values)   # For each vector size
    for j in 1:length(q_values)  # For each key numbers
        time_optimized_search[i, j] = dummy_search(optimized_search, vector_list[i], key_list[j])
        end
    end
end

LoadError: MethodError: no method matching isless(::Vector{Int64}, ::Int64)

[0mClosest candidates are:
[0m  isless([91m::Missing[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:87[24m[39m
[0m  isless(::Any, [91m::Missing[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:88[24m[39m
[0m  isless([91m::AbstractFloat[39m, ::Real)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:179[24m[39m
[0m  ...


In [ ]:
# Binary search - Benchmarking
for i in 1:length(n_values)   # For each vector size
    for j in 1:length(q_values)  # For each key numbers
        time_binary_search[i, j] = dummy_search(binary_search, vector_list[i], key_list[j])
    end
end

In [ ]:
# Ploting graphics
for j in 1:length(q_values)
    p = plot(
        n_values,
        time_simple_search[:, j],
        label = "Simples",
        xlabel = "Tamanho do Vetor (n)",
        ylabel = "Tempo (s)",
        title = "Numero de Buscas = $(q_values[j])",
        legend = :topleft,
        lw = 2  # Line Width
    )
    plot!(n_values, time_optimized_search[:, j], label = "Otimizada", lw = 2)
    plot!(n_values, time_binary_search[:, j], label = "Binária", lw = 2)
    plot!(n_values, time_to_sort, label = "Ordenação", lw = 2)

    display(p)
    savefig(p, "../imgs/benchmark_q_$(q_values[j]).png")
end